In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import bisplrep, bisplev
from scipy import integrate

In [57]:
def create_knots(k,n,min=0,max=10):
    # Create open-uniform knot vector for a B-spline of degree k and with n+1 control points between the min and max values.  
    return np.concatenate(
        [(min)*np.zeros(k),
         np.linspace(0,max,n),
         (max)*np.ones(k)]
        )    

def fit_Bspline(x,y,z,k,tx,ty):
    #Fit B-spline on the given dataset of x,y,z and return its tck form and the total-error.
    tck,total_error,_,_=bisplrep(x, y, z, kx=k, ky=k, task=-1, s=0, tx=tx, ty=ty, full_output=1)
    depth_loss = 1/(len(z))*total_error #divide by the number of datapoints to get the mean-squared error (MSE).
    return tck, depth_loss

def curvature_loss(tck, x_i, y_i): 
    #Discrete approximation of integrating the second derivatives of the B-spline (parameterized by tck) w.r.t. x and y over the domain of [min_x,max_x] x [min_y,max_y]
    #Computing a double integral is rather computationally expensive, therefore, the second partial derivatives are evaluated and summed over all datapoints.
    
    #Absolute value of the second derivative of the b-spline w.r.t. x and y respectively
    bs_xx = lambda x,y: np.abs(bisplev(x,y,tck,2,0)) 
    bs_yy = lambda x,y: np.abs(bisplev(x,y,tck,0,2))

    #Evaluate the absolute value of the second derivatives at all the datapoints (x_i,y_i) and sum them. This gives an approximation of the total curvature
    total_curv = np.sum(bs_xx(x_i,y_i)+bs_yy(x_i,y_i)) 

    #Obtain the loss as the total curvature dived by the number of datapoints
    curv_loss = (1/(len(x_i)*len(y_i)))*total_curv
    return curv_loss

def curvature_loss_int(tck, min_x=0, max_x=10, min_y=0, max_y=10): 
    #Integrate the second derivative of the B-spline, parameterized by tck, w.r.t. x and y over the domain of [min_x,max_x] x [min_y,max_y]
    #Computing a double integral is rather computationally expensive, therefore it is suggested to do this computations on a HPC
    
    #Absolute value of the second derivative of the b-spline w.r.t. x and y respectively
    bs_xx = lambda x,y: np.abs(bisplev(x,y,tck,2,0)) 
    bs_yy = lambda x,y: np.abs(bisplev(x,y,tck,0,2))

    #Integrate the absolute value of the second derivatives over the whole x and y domain
    curv_loss_x,_ = integrate.dblquad(bs_xx,min_x,max_x,min_y,max_y) 
    curv_loss_y,_ = integrate.dblquad(bs_yy,min_x,max_x,min_y,max_y)

    #Obtain the loss as the total as the sum of the two integrals divided over the domain ((max_x-min_x)*(max_y-min_y))
    curv_loss = (1/((max_x-min_x)*(max_y-min_y)))*(curv_loss_x + curv_loss_y)
    return curv_loss
    
def total_loss(depth_loss, curv_loss, kap_1):
    #Assert that all the weights (kap_1,...,kap_n) sum up to unity.
    return kap_1*depth_loss + (1-kap_1)*curv_loss

In [56]:
#This is just an example.

#make it a nice wave function (imre) and visualize it. 
#Generate random data points
np.random.seed(42)  # For reproducibility
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
z = np.random.random((100, 100))

# Create a meshgrid from x and y
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid arrays and z array to pass to bisplrep
x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = z.flatten()

# Obtain the border values of the domain. 
max_x=np.max(x_flat)
min_x=np.min(x_flat)
max_y=np.max(y_flat)
min_y=np.min(y_flat)

#Define the parameters
k =3 #Degree (both x and y direction)
n = 10 # The number of control points (n+1) 

# Create open-uniform knots over the domain.
tx=create_knots(k,n, min_x, max_x)
ty=create_knots(k,n, min_y, max_y)

#Create a B-Spline representation using previous determined knots on the data and store the depthmap losserror. 
tck,dm_loss=fit_Bspline(x_flat, y_flat, z_flat, k=k, tx=tx, ty=ty)

#Compute the curvature loss
curv_loss=curvature_loss(tck, x, y) # approximation
curv_loss_int=curvature_loss_int(tck)

#Define arbritary weigths for each loss to contribute in the total loss.
kap_1=0.5
total_loss = total_loss(dm_loss,curv_loss,kap_1)

#print the losses
print("Depthmap loss:", dm_loss)
print("Curvature loss:", curv_loss)
print("Curv loss int:", curv_loss)
print("Total loss:", total_loss)


Depthmap loss: 0.08161902474992237
Curvature loss: 0.32659768521330723
Total loss: 0.2041083549816148


In [53]:
bs_xx = lambda x,y: np.abs(bisplev(x,y,tck,2,0)) 
np.size(bs_xx(x,y))

10000